In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [54]:
# !pip install peft
# !pip install accelerate
# !pip install bitsandBytes
# !pip install transformers
# !pip install datasets

In [55]:
# !pip install GPUtil

In [4]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

2025-06-06 06:28:22.793145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749191302.976429      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749191303.030729      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
# for kaggle noebook only
!git config --global credential.helper store

import os
from huggingface_hub import notebook_login

if "COLAB_GPU" in os.environ:
    # For Google Colab
    !huggingface-cli login
else:
    # For local Jupyter or other environments
    notebook_login()

In [73]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

In [16]:
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [57]:
train_dataset = load_dataset("QuyenAnhDE/Diseases_Symptoms", split="train")

README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [59]:
train_dataset[0]

{'Code': 1,
 'Name': 'Panic disorder',
 'Symptoms': 'Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness',
 'Treatments': 'Antidepressant medications, Cognitive Behavioral Therapy, Relaxation Techniques'}

In [61]:
train_dataset[1]

{'Code': 2,
 'Name': 'Vocal cord polyp',
 'Symptoms': 'Hoarseness, Vocal Changes, Vocal Fatigue',
 'Treatments': 'Voice Rest, Speech Therapy, Surgical Removal'}

In [62]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    use_fast=True,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [63]:
tokenized_train_dataset = []

for item in train_dataset:
    prompt = f"""### Instruction:\nProvide medical information about the disorder.\n\n### Input:\nName: {item['Name']}\nSymptoms: {item['Symptoms']}\n\n### Response:\nTreatments: {item['Treatments']}"""
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized_train_dataset.append(tokenized)

In [64]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Convert list of tokenized dicts to a Dataset object
tokenized_train_dataset = Dataset.from_list(tokenized_train_dataset)

In [67]:
print(tokenized_train_dataset[1].keys())  # Shows all keys

# To see input_ids or attention_mask separately:
print(tokenized_train_dataset[1]['input_ids'])
print(tokenized_train_dataset[1]['attention_mask'])

dict_keys(['input_ids', 'attention_mask'])
[128000, 14711, 30151, 512, 61524, 6593, 2038, 922, 279, 19823, 382, 14711, 5688, 512, 678, 25, 98403, 23125, 1499, 1100, 198, 29012, 52273, 25, 17723, 1590, 24639, 11, 98403, 29240, 11, 98403, 26417, 22711, 271, 14711, 6075, 512, 51, 1244, 1392, 25, 29030, 9240, 11, 39841, 40143, 11, 71212, 57817, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 1

In [66]:
tokenizer.eos_token

'<|eot_id|>'

In [68]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [69]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetunedModel",
    run_name="llama-finetune-medical",
    per_device_train_batch_size=4,  # increase batch size if GPU memory allows
    gradient_accumulation_steps=1,  # reduce accumulation for speed
    num_train_epochs=3,
    learning_rate=1e-4,
    bf16=False,
    fp16=True,  # enable mixed precision if supported
    optim="paged_adamw_8bit",
    logging_dir="./log",
    save_strategy="epoch",
    save_steps=50,
    logging_steps=10,
    report_to="none",
    max_steps=-1,
    load_best_model_at_end=False,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model.config.use_cache = False  # required for gradient checkpointing

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    data_collator=data_collator,
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.324800
20,0.954700
30,0.895800
40,0.758600
50,0.803100
60,0.837300
70,0.823200
80,0.770800
90,0.816100
100,0.813600


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=300, training_loss=0.587778073946635, metrics={'train_runtime': 2088.2152, 'train_samples_per_second': 0.575, 'train_steps_per_second': 0.144, 'total_flos': 1.0435845685248e+16, 'train_loss': 0.587778073946635, 'epoch': 3.0})

In [71]:
model.save_pretrained("./finetunedModel")
tokenizer.save_pretrained("./finetunedModel")

('./finetunedModel/tokenizer_config.json',
 './finetunedModel/special_tokens_map.json',
 './finetunedModel/tokenizer.json')

In [74]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./finetunedModel")
tokenizer = AutoTokenizer.from_pretrained("./finetunedModel")

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [70]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-3.2-3B-Instruct"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
  )


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizer'.


TypeError: not a string

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True

                              )

modelFinetuned = PeftModel.from_pretrained(base_model, "finetunedModel/checkpoint-20")

In [ ]:
user_question = "When did Hawaii wildfires start?"

eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()